In [1]:
#from mvp, build and run pipeline to generate .py files in src folder
#%%writefile src/obtain.py

# write functions that form a data pipeline, 
# then run this cell to save them in src/obtain.py

#use this at the end of the project to generate py files, import and run them from a main ftn for cleaner code

UsageError: Line magic function `%%writefile` not found.


In [2]:
# import data pipeline functions from obtain.py
# def run_data_pipeline():
#     # call data pipeline functions in order
#     pass

# run_data_pipeline()

## Loading and cleaning text data
### Import libraries

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import nltk
import spacy #for faster tokenization and lemmatization

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import re

%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Loading small dataset 
Kaggle 2MB What's Cooking data set - features: id, ingredients, cuisine type (target)

Use it to test cusine type classification. Should be able to use the best model on larger RecipeBox dataset after extracting ingredients.

*is there a way to out put probabilities of cuisine types? otherwise wouldn't get recipes from diff cuisines with the same ingredients* - predict_proba? or topic modelling and cluster by ingredient

In [6]:
cdata = pd.read_json (r'/Users/xinrucheng/Documents/GitHub/metisproject04/data/raw/whats-cooking/train.json')

In [8]:
cdata.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [7]:
cdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 3 columns):
id             39774 non-null int64
cuisine        39774 non-null object
ingredients    39774 non-null object
dtypes: int64(1), object(2)
memory usage: 932.3+ KB


The small dataset has 39774 observations, and 3 features (id, cuisine, ingredients). We want to use it to build a cuisine-type classifier based on ingredients, and then hopefully apply it to the larger recipe box dataset (125000 observations). The source for the recipe box dataset includes cuisine type in tags, but the tags were not scraped (still have the option of scraping it if small classifier doesn't work?)

In [13]:
cdata['cuisine'].nunique()

20

So there are no null values, and there are 20 cuisine types for 39774 recipes in this dataset. That seems like it should be sufficient as a classifier for the larger dataset (can improve upon it later with scraped tags data).

In [14]:
cdata['cuisine'].unique()

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

Looking at the 20 cuisine types in this data set, I think "british" and "irish" are a bit redundant (would've liked to have German instead of one of those; also mexican and cajun_creole probably have some overlaps also). I'll work with this for now, keeping in mind topic modelling and clustering on the bigger data set might lead to a different number of clusters.

In [ ]:
#also recipe box repo has scraping code, if need to scrape more observations or tags

#before proceeding with the classifier, quickly inspect the recipe box data:

### Loading the RecipeBox dataset
#### allrecipes

In [15]:
#the three files are scraped from foodnetwork, epicurious, and allrecipes.com
#start with allrecipes.com - took 18.4s to load, can consider putting it into database?
rbdata_ar = pd.read_json(r'/Users/xinrucheng/Documents/GitHub/metisproject04/data/raw/recipes_raw/recipes_raw_nosource_ar.json')

In [16]:
rbdata_ar.head() #need to swap rows and columns
#features are title, ingredients, instructions, and picture_link(not needed?)

,rmK12Uau.ntP510KeImX506H6Mr6jTu,5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,clyYQv.CplpwJtjNaFGhx0VilNYqRxu,BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,kq.naD.8G19M4UU9dVvJgHtpfo.l/eC,lYrgWNn00EXblOupzM3tL0jGr9O0CB2,Fu0DgGYFUGwc0BBlN6r20o/ihOVs5bO,MBRNtqELRRuv8zJH4k7Aba2bmIc2A3C,ZPyPoMiNvgAfrKcRpH9FEYV/XsPZBsW,...,uHHb42/tuIKsmN5U6l9AD.FdVpSFxs6,IPAeN3L6rm2oughJpUhbG038k.ACJ0K,3UIhlTQFH5jyIaHN8zeKlK5V.94Kjwu,PdBxkE2gnI/.ynokkp1Hu1KLGZnGdei,SB46Udqc5Svsi70S1qRmRLv5tlg8Oca,gehEOcDPtU3SmNSXrwWwWD4ulPpUdMO,VRAsyF.1xMBYqAVKX1biyIORH6N6qzy,Lf8/u.0k2029QMSQFrHS4gRsvKOQFUG,ay.AqX/9ysBtWHcnHoDeGAyJ5Orla8e,2Q3Zpfgt/PUwn1YABjJ5A9T3ZW8xwVa
title,Slow Cooker Chicken and Dumplings,Awesome Slow Cooker Pot Roast,Brown Sugar Meatloaf,Best Chocolate Chip Cookies,Homemade Mac and Cheese Casserole,Banana Banana Bread,Chef John's Fisherman's Pie,Mom's Zucchini Bread,The Best Rolled Sugar Cookies,Singapore Chili Crabs,...,Citrus Creme Brulee,Honey Nutty Granola,Salmon en Croute,Homemade Blender Butter,Gluten Free Chicken Noodle Soup,Thai-Indian Veggie Soup,Coconut Milk-Free Panang Curry Chicken,Cooked Cold Salad,Easy Eggnog Creme Brulee,Super Power Stovetop Granola
ingredients,"[4 skinless, boneless chicken breast halves AD...",[2 (10.75 ounce) cans condensed cream of mushr...,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...","[1 cup butter, softened ADVERTISEMENT, 1 cup w...",[8 ounces whole wheat rotini pasta ADVERTISEME...,"[2 cups all-purpose flour ADVERTISEMENT, 1 tea...","[For potato crust: ADVERTISEMENT, 3 russet pot...","[3 cups all-purpose flour ADVERTISEMENT, 1 tea...","[1 1/2 cups butter, softened ADVERTISEMENT, 2 ...","[Sauce: ADVERTISEMENT, 1/2 cup ketchup ADVERTI...",...,"[2 oranges, juiced ADVERTISEMENT, lemon, juice...","[3 cups rolled oats ADVERTISEMENT, 1 1/2 cups ...","[1 cup watercress, or as desired ADVERTISEMENT...","[2 pints heavy whipping cream ADVERTISEMENT, s...",[1/2 (12 ounce) box Barilla® Gluten Free Elbow...,"[2 teaspoons olive oil ADVERTISEMENT, 1/4 cup ...","[2 cups light cream ADVERTISEMENT, 1/4 teaspoo...","[3 tablespoons bacon grease ADVERTISEMENT, 2 c...","[4 egg yolks ADVERTISEMENT, 1 tablespoon white...","[1/4 cup canola oil ADVERTISEMENT, 3 cups quic..."
instructions,"Place the chicken, butter, soup, and onion in ...","In a slow cooker, mix cream of mushroom soup, ...",Preheat oven to 350 degrees F (175 degrees C)....,Preheat oven to 350 degrees F (175 degrees C)....,Preheat oven to 350 degrees F. Line a 2-quart ...,Preheat oven to 350 degrees F (175 degrees C)....,Bring a large saucepan of salted water and to ...,Grease and flour two 8 x 4 inch pans. Preheat ...,"In a large bowl, cream together butter and sug...","Whisk ketchup, chicken broth, egg, soy sauce, ...",...,Preheat oven to 300 degrees F (150 degrees C)....,Preheat oven to 300 degrees F (150 degrees C)....,Preheat oven to 375 degrees F (190 degrees C)....,Pour cream into a blender. Cover and blend unt...,Saute onions in olive oil over medium heat unt...,Heat oil in a large pot over medium heat. Add ...,Heat cream and coconut extract in a skillet or...,Heat bacon grease in a skillet over medium-hig...,Preheat oven to 350 degrees F (175 degrees C)....,Heat 1/4 cup canola oil in large skillet over ...
picture_link,55lznCYBbs2mT8BTx6BTkLhynGHzM.S,QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,YCnbhplMgiraW4rUXcybgSEZinSgljm,jRnWGDXDdyOg3rta4/HVAR2rD19XubC,aUca10AaD8T2yYvcLOgH/UJlR5/OhOe,YdgEVyLVffZgh9NZPN3Eqj6MaX8KdzK,UrgvDGu4roLiho160fTVIwCUrGZna8i,OFp6yXFwzlrkMQ5STffYPllxQvMVLUS,...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,cWEzUSv9Ozr3b4MxNVCqJYgTjIS.kHm,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,qE58a7Z1Au0GXvPO188iHZZVqna9hLa


In [22]:
#try out "orient" argument on smaller dataset, want to swap rows and columns:
#pd.read_json(r'/Users/xinrucheng/Documents/GitHub/metisproject04/data/raw/whats-cooking/train.json', orient='split')

In [23]:
rb_ar=rbdata_ar.T #can just do pandas transpose, don't need to set orient parameter in read_json!
rb_ar.head()

,title,ingredients,instructions,picture_link
rmK12Uau.ntP510KeImX506H6Mr6jTu,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves AD...","Place the chicken, butter, soup, and onion in ...",55lznCYBbs2mT8BTx6BTkLhynGHzM.S
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ...",QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe
clyYQv.CplpwJtjNaFGhx0VilNYqRxu,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56
BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi
N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...,YCnbhplMgiraW4rUXcybgSEZinSgljm


In [24]:
#*can I do something more interesting with the ingredient and instructions data to access recipe similarity? (difficulty already used)
#clusters can be diff courses of meal? main, dessert; price? prep time?
#how to use links??

In [25]:
rb_ar.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39802 entries, rmK12Uau.ntP510KeImX506H6Mr6jTu to 2Q3Zpfgt/PUwn1YABjJ5A9T3ZW8xwVa
Data columns (total 4 columns):
title           39522 non-null object
ingredients     39522 non-null object
instructions    39522 non-null object
picture_link    39522 non-null object
dtypes: object(4)
memory usage: 2.8+ MB


The allrecipes.com portion of the RecipeBox dataset has 39522 recipes, similar in size to the small what's cooking dataset! Good for testing out classifer. Because total was ~125000, so that's about 1/3 of total (can check, the other 2 should be the same size)

In [ ]:
#*total length 39802 observations, but only 39522 non-null for each feature? Need to drop rows with missing values


#### epicurious

In [28]:
rbdata_epi = pd.read_json(r'/Users/xinrucheng/Documents/GitHub/metisproject04/data/raw/recipes_raw/recipes_raw_nosource_epi.json')

In [30]:
rb_epi=rbdata_epi.T

In [31]:
rb_epi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25323 entries, 05zEpbSqcs9E0rcnCJWyZ9OgdH0MLby to qznqHiNpq0AB1AYn002A2HvaEFnN0lq
Data columns (total 4 columns):
ingredients     25323 non-null object
picture_link    12119 non-null object
instructions    25323 non-null object
title           25323 non-null object
dtypes: object(4)
memory usage: 2.2+ MB


In [36]:
#only about half of the epicurious dataset has pictures, also this is the smallest dataset. 
#the columns are also in a different order
rb_epi.head()

,ingredients,picture_link,instructions,title
05zEpbSqcs9E0rcnCJWyZ9OgdH0MLby,"[12 egg whites, 12 egg yolks, 1 1/2 cups sugar...",None,"Beat the egg whites until stiff, gradually add...",Christmas Eggnog
mF5SZmoqxF4WtIlhLRvzuKk.z6s7P2S,"[18 fresh chestnuts, 2 1/2 pounds veal stew me...",None,Preheat oven to 400°F. Using small sharp knife...,"Veal, Carrot and Chestnut Ragoût"
oQV5D7cVbCFwmrDs3pBUv2y.AG0WV26,"[2 tablespoons unsalted butter, softened, 4 or...",3xjktRst3I5lDZ2Z5kTOtqQyzZFFN9u,Preheat the oven to 350°F. Spread the softened...,Caramelized Bread Pudding with Chocolate and C...
Z9seBJWaB5NkSp4DQHDnCAUBTwov/1u,"[3/4 pound Stilton, crumbled (about 3 cups) an...",None,"In a food processor blend the Stilton, the cre...",Sherried Stilton and Green Peppercorn Spread
bB3GxoAplVZeoX3fzWNWyeECtQFxw6G,"[2 cups (about 9 1/2 ounces) whole almonds, to...",None,Position rack in center of oven and preheat to...,Almond-Chocolate Macaroons


#### foodnetwork

In [37]:
rbdata_fn = pd.read_json(r'/Users/xinrucheng/Documents/GitHub/metisproject04/data/raw/recipes_raw/recipes_raw_nosource_fn.json')

In [38]:
rb_fn=rbdata_fn.T

In [39]:
rb_fn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60039 entries, p3pKOD6jIHEcjf20CCXohP8uqkG5dGi to 7cXA77UpdDtIfBug2v6lEVIuV3Zcvhm
Data columns (total 4 columns):
instructions    59628 non-null object
ingredients     59802 non-null object
title           59750 non-null object
picture_link    30952 non-null object
dtypes: object(4)
memory usage: 4.8+ MB


This is the largest single-source dataset with 60039 entries, but it has many missing values. 
What could cause the most issues for out purposes is missing title and ingredients. It would make sense to drop the rows that do not have title or ingredients, and even the ones missing instructions. We won't likely need the picture link, so we can drop those columns for all three files in the recipebox dataset.

In [40]:
rb_fn.head()

,instructions,ingredients,title,picture_link
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,Toss ingredients lightly and spoon into a butt...,"[1/2 cup celery, finely chopped, 1 small green...",Grammie Hamblet's Deviled Crab,None
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,Watch how to make this recipe.\nSprinkle the s...,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Infineon Raceway Baked Beans,Ja5uaD8Q7m7vvtWwk2.48dr1eCre/qi
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,"In a large saucepan, let the beans soak in eno...","[1 1/2 cups dried black beans, picked over and...",Southwestern Black Bean Dip,None
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,Watch how to make this recipe.\nPreheat the ov...,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Sour Cream Noodle Bake,nm/WxalB.VjEZSa0iX9RuZ8xI51Y7bS
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,Special equipment: sushi mat\nCook the brown r...,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Sushi Renovation,None


### Data cleaning

In [ ]:
#drop id columns in what's cooking dataset, drop picture_link columns in all 3 recipebox datasets

#also drop missing values in foodnetwork and allrecipes data. wsc and epicurious do not have missing values

#### unigram/bigram/trigrams? 
what is it for? how to use?

#### Stemming and Lemmatization?
review NLP overview lecture?

## Vectorization
### Count Vectorizer

### TF-IDF

## Dimensionality Reduction

### Topic Modelling

In [41]:
#LSA/NMF on wsc data before classfier?

## Clustering

In [42]:
#on both wsc and rb datasets?

## Recommendation System

### Ingredient-similarity
### Flavor-similarity**